# Temel Veri Ön İşleme (Data Preprocessing) – Teorik + Uygulamalı Rehber

Hoş geldiniz! 👋

Bu not defteri, Veri Bilimi ve Makine Öğrenmesi (Machine Learning) dünyasına adım atanlar için hazırlanmıştır. Hiçbir ön bilginiz olmasa bile, adım adım ilerleyerek ham veriyi nasıl analize hazır hale getireceğinizi öğreneceksiniz.

## 🎯 Bu Rehberde Neler Öğreneceksiniz?
1. **Veri Ön İşleme Nedir?** Neden bu kadar önemli?
2. **Veriyi Yükleme ve İnceleme:** Verimiz neye benziyor?
3. **Keşifçi Veri Analizi (EDA):** Veriyi görselleştirme ve anlama.
4. **Eksik Veriler (Missing Values):** Kayıp verilerle nasıl başa çıkarız?
5. **Aykırı Değerler (Outliers):** Aşırı uç değerleri nasıl tespit ederiz?
6. **Kategorik Değişkenler:** Yazı (string) verileri sayılara çevirme.
7. **Özellik Ölçeklendirme (Scaling):** Verileri aynı ölçeğe getirme.
8. **Özellik Mühendisliği (Feature Engineering):** Yeni özellikler türetme.
9. **Eğitim/Test Ayrımı (Train/Test Split):** Modelimizi nasıl test edeceğiz?
10. **Tüm Süreç (Pipeline):** Hepsini birleştirelim.

---

## BÖLÜM 1: Giriş – Veri Ön İşleme Nedir?

Bir yemek yaptığınızı düşünün. 🍳 En iyi yemeği yapmak için önce malzemeleri yıkamanız, soymanız ve doğramanız gerekir. Çürük domatesleri atar, patatesleri soyarsınız. İşte **Veri Ön İşleme** de budur.

Makine öğrenmesi modelleri matematikle çalışır. Kirli, eksik veya düzensiz veriyi bu modellere verirseniz, sonuç da kötü olur.

> **"Garbage in, garbage out"** (Çöp girerse, çöp çıkar) ilkesi buradan gelir.

Hadi başlayalım! 🚀

## BÖLÜM 2: Veri Setini Tanıyalım

Bu çalışma için hazırladığımız `dataset.csv` dosyasını kullanacağız. Bu verisetinde ev fiyatları ve evlerin bazı özellikleri bulunuyor.

İlk işimiz kütüphanesi çağırmak ve veriyi okumak.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Uyarıları kapatalım (daha temiz bir görüntü için)
import warnings
warnings.filterwarnings('ignore')

# Veriyi yükleyelim
df = pd.read_csv("dataset.csv")

# İlk 5 satırı görelim
df.head()

**Çıktıyı Yorumlayalım:**
- `Ev_Buyuklugu`: Evin metrekaresi (m²)
- `Oda_Sayisi`: Odaların sayısı
- `Konum`: Evin bulunduğu bölge (Merkez, Banliyö, Kırsal)
- `Bina_Yasi`: Binanın yaşı
- `Fiyat`: Evin satış fiyatı (Hedef değişkenimiz / Target)

## BÖLÜM 3: Keşifçi Veri Analizi (Exploratory Data Analysis - EDA)

Veriyi temizlemeye başlamadan önce, neyle karşı karşıya olduğumuzu anlamalıyız. Doktorun ameliyattan önce röntgen çekmesi gibidir. 🩺

In [ ]:
# Veri seti hakkında genel bilgi (Satır sayısı, sütun tipleri, boş değerler)
print("--- Veri Seti Bilgisi ---")
df.info()

print("\n--- İstatistiksel Özet ---")
# İstatistiksel özet (Ortalama, standart sapma, min, max)
df.describe().T

### Eksik Verileri Görelim
Hangi sütunda kaç tane eksik (NaN - Not a Number) değer var?

In [ ]:
df.isnull().sum()

Görünüşe göre bazı sütunlarda eksik verilerimiz var. Bunları birazdan halledeceğiz.

### Görselleştirme
Fiyatların dağılımına bakalım.

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df['Fiyat'], kde=True)
plt.title('Ev Fiyatları Dağılımı')
plt.show()

## BÖLÜM 4: Eksik Veriler (Missing Values) 🕳

Eksik veriler modelleri bozar. Onlarla başa çıkmak için stratejilerimiz şunlardır:

1. **Silmek (Dropping):** Veri çok fazlaysa ve eksik kısım azsa silinebilir.
2. **Doldurmak (Imputation):** Ortalama (mean), Medyan (ortanca) veya Mod (en sık tekrar eden) ile doldurulabilir.

Bizim verimizde az satır olduğu için **doldurma** yöntemini seçeceğiz.

In [ ]:
# Önce eksik verilerin olduğu satırları bir görelim
print(df[df.isnull().any(axis=1)])

# Stratejimiz:
# - 'Bina_Yasi' sayısal bir değer -> ORTALAMA ile dolduralım.
# - 'Konum' kategorik bir değer -> MOD (en sık geçen) ile dolduralım.
# - 'Ev_Buyuklugu' sayısal -> MEDYAN ile dolduralım (Aykırı değerlerden etkilenmez)

# 1. Bina Yaşı (Ortalama)
yas_ortalamasi = df['Bina_Yasi'].mean()
df['Bina_Yasi'].fillna(yas_ortalamasi, inplace=True)

# 2. Ev Büyüklüğü (Medyan)
buyukluk_medyan = df['Ev_Buyuklugu'].median()
df['Ev_Buyuklugu'].fillna(buyukluk_medyan, inplace=True)

# 3. Konum (Mod)
konum_modu = df['Konum'].mode()[0]
df['Konum'].fillna(konum_modu, inplace=True)

# Kontrol edelim
df.isnull().sum()

Harika! Artık eksik verimiz kalmadı. ✅

## BÖLÜM 5: Aykırı Değerler (Outliers) 🚨

Aykırı değerler, genel örüntüye uymayan aşırı uç değerlerdir. (Örneğin: Mahalledeki evlerin ortalaması 3 milyon TL iken, bir evin 100 milyon TL olması).

Aykırı değerleri tespit etmek için **IQR (Interquartile Range)** yöntemini kullanabiliriz.

In [ ]:
# Fiyat sütunu için aykırı değerlere bakalım
sns.boxplot(x=df['Fiyat'])
plt.title('Fiyat Aykırı Değer Kontrolü (Boxplot)')
plt.show()

Grafikteki kutunun sağındaki tek nokta muhtemelen bir aykırı değerdir. Hadi onu sayılarla bulalım.

In [ ]:
Q1 = df['Fiyat'].quantile(0.25)
Q3 = df['Fiyat'].quantile(0.75)
IQR = Q3 - Q1

alt_sinir = Q1 - 1.5 * IQR
ust_sinir = Q3 + 1.5 * IQR

print(f"Alt Sınır: {alt_sinir}")
print(f"Üst Sınır: {ust_sinir}")

# Aykırı değerleri görelim
aykirilar = df[(df['Fiyat'] < alt_sinir) | (df['Fiyat'] > ust_sinir)]
print("\nAykırı Değerler:")
print(aykirilar)

Bu aykırı değeri verisetinden çıkarabiliriz veya baskılayabiliriz. Bu örnekte çıkaralım.

In [ ]:
df = df[(df['Fiyat'] >= alt_sinir) & (df['Fiyat'] <= ust_sinir)]

# İndeksleri sıfırlayalım
df.reset_index(drop=True, inplace=True)
print("Aykırı değerler temizlendi. Yeni satır sayısı:", len(df))

## BÖLÜM 6: Kategorik Değişken Dönüşümü (Encoding) 🏷

Bilgisayarlar kelimelerden anlamaz, sayılardan anlar. `Konum` sütunumuzda "Merkez", "Kırsal" gibi yazılar var. Bunları sayıya çevirmeliyiz.

İki temel yöntem vardır:
1. **Label Encoding:** Her kategoriye bir sayı verir (Merkez=0, Kırsal=1...)
2. **One-Hot Encoding:** Her kategori için yeni sütun açar (Konum_Merkez: 1 veya 0)

Genellikle sıralama önemsizse (nominal) **One-Hot Encoding** tercih edilir.

In [ ]:
# Konum sütununa One-Hot Encoding uygulayalım
df_encoded = pd.get_dummies(df, columns=['Konum'], drop_first=True)

print("Eski Sütunlar:", df.columns.tolist())
print("Yeni Sütunlar:", df_encoded.columns.tolist())
df_encoded.head()

Not: `drop_first=True` diyerek bir sütunu attık (Dummy Variable Trap'ten kaçınmak için). Örneğin hem Merkez hem Banliyö 0 ise, zaten Kırsal demektir.

## BÖLÜM 7: Özellik Ölçeklendirme (Feature Scaling) ⚖

Bakın: `Fiyat` sütunu milyonlarla ifade edilirken, `Oda_Sayisi` 3-5 gibi küçük sayılar. Makine öğrenmesi modelleri büyük sayıları daha "önemli" sanabilir. Bunu engellemek için hepsini aynı ölçeğe (genelde -1 ile 1 arasına veya 0 ile 1 arasına) getiririz.

**StandardScaler (Standartlaştırma):** Ortalamayı 0, varyansı 1 yapar.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Ölçeklenecek sütunlar (Sadece sayısal ve target olmayanlar)
cols_to_scale = ['Ev_Buyuklugu', 'Bina_Yasi']

# Kopyasını alıp işlem yapalım
df_scaled = df_encoded.copy()

df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

df_scaled.head()

Gördüğünüz gibi değerler artık birbirine daha yakın aralıklarda.

## BÖLÜM 8: Özellik Mühendisliği (Feature Engineering) 💡

Var olan veriden, modelin daha iyi öğrenebileceği *yeni* veriler türetme sanatıdır.

Örnek: `Oda_Sayisi` ve `Ev_Buyuklugu` var. Acaba "Oda başına düşen metrekare" fiyatı etkiler mi?

In [ ]:
# Orijinal veriye dönelim (okunması kolay olsun diye)
df['Oda_Basina_M2'] = df['Ev_Buyuklugu'] / df['Oda_Sayisi']

df[['Ev_Buyuklugu', 'Oda_Sayisi', 'Oda_Basina_M2']].head()

## BÖLÜM 9: Eğitim ve Test Ayrımı (Train / Test Split) 🧩

Modeli eğittikten sonra ne kadar başarılı olduğunu görmek için, verinin bir kısmını "saklarız". Modeli verinin %80'i ile eğitir, kalan %20'si ile test ederiz (sınav yaparız).

Bunun için `sklearn` kütüphanesini kullanacağız.

In [ ]:
from sklearn.model_selection import train_test_split

# Hedef (Target) ve Özellikler (Features) ayrımı
X = df_encoded.drop('Fiyat', axis=1)  # Fiyat hariç her şey girdi
y = df_encoded['Fiyat']               # Fiyat hedef

# Veriyi bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti Boyutu: {X_train.shape}")
print(f"Test Seti Boyutu: {X_test.shape}")

## BÖLÜM 10: Mini Pipeline (Özet Akış)

Şimdiye kadar parça parça yaptıklarımızı tek bir akışta görelim.

In [ ]:
# 1. Veriyi Yükle
data = pd.read_csv("dataset.csv")

# 2. Eksikleri Doldur
data['Bina_Yasi'].fillna(data['Bina_Yasi'].mean(), inplace=True)
data['Konum'].fillna(data['Konum'].mode()[0], inplace=True)
data['Ev_Buyuklugu'].fillna(data['Ev_Buyuklugu'].median(), inplace=True)

# 3. Aykırı Değerleri At (Basitçe)
Q1 = data['Fiyat'].quantile(0.25)
Q3 = data['Fiyat'].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data['Fiyat'] < (Q1 - 1.5 * IQR)) | (data['Fiyat'] > (Q3 + 1.5 * IQR)))]

# 4. Encoding
data = pd.get_dummies(data, columns=['Konum'], drop_first=True)

# 5. Split
X = data.drop('Fiyat', axis=1)
y = data['Fiyat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Scaling (Split'ten SONRA yapılır! Data Leakage olmaması için)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Pipeline tamamlandı. Model eğitimine hazır!")

## BÖLÜM 11: Yaygın Başlangıç Hataları ⚠️

1. **Veri Sızıntısı (Data Leakage):** Scaler işlemini tüm veriye yapıp sonra bölerseniz, test verisindeki bilgiyi eğitim setine sızdırmış olursunuz. Önce bölün, sonra X_train üzerinden `fit` yapın.
2. **Aşırı Temizlik:** Bazen eksik görünen veriler aslında bir bilgi taşır. Her şeyi silmeyin.
3. **Kategorik Veriyi Unutmak:** Modeller yazı tipi veriyi kabul etmez, mutlaka encode edin.

## BÖLÜM 12: Egzersizler 💪

Sıra sizde! Aşağıdaki görevleri yapmayı deneyin:

1. `dataset.csv` dosyasını tekrar yükleyin.
2. Eksik değerleri ortalama yerine 0 ile doldurun.
3. `Oda_Sayisi` özelliğini kullanarak yeni bir özellik türetin (örn: Oda sayısı karesi).
4. `StandardScaler` yerine `MinMaxScaler` kullanmayı deneyin.

Başarılar! 🎓

---